In [4]:
from utils import *

In [2]:
from prepare_dataset import prepare_dataset
import torch.utils.data as data_utils

data = prepare_dataset()
train_set = data[0]
valid_set = data[1]

train_loader = data_utils.DataLoader(train_set, batch_size=64, pin_memory=True, shuffle=True)
valid_loader = data_utils.DataLoader(valid_set, batch_size=64,)

from torch.utils.data import ConcatDataset

dataloaders = {'train':train_loader,'valid':valid_loader}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'valid']}
dataset = ConcatDataset([train_set, valid_set])

[1 1 1 ... 1 1 1]
Number of positive samples in training data: 1605 (50.03% of total)
Number of positive samples in validation data: 395 (49.87% of total)


In [3]:
from sklearn.model_selection import KFold
from model import EfficientNet
import torch
from tqdm import tqdm
from torch.autograd import Variable
device = torch.device("mps")

kfold = KFold(n_splits=5, shuffle=True)
for fold, (train_idx, valid_idx) in enumerate(kfold.split(dataset)):

    model = EfficientNet.from_name('efficientnet-phospho-B-15')
    model = model.to(device)
   # Define data loaders for training and testing data in this fold
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    print(len(train_idx))
    print(len(valid_idx))
    test_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                dataset,
                batch_size=64, sampler=train_subsampler)
    validloader = torch.utils.data.DataLoader(
                dataset,
                batch_size=64, sampler=test_subsampler)

    for epoch in tqdm(range(20), position=0, leave=True):
        # print('-' * 60)
        # print('Epoch {}/{}'.format(epoch+1, 20))

        train_corrects = 0.0
        train_loss = 0.0
        train_precision, train_recall, train_f1 = 0.0, 0.0, 0.0

        for _, (inputs, labels) in enumerate(tqdm(trainloader, position=1, leave=True)):
                model.train(True)
                inputs = Variable(inputs.to(device, dtype=torch.float), requires_grad=True)
                labels = Variable(labels.to(device))
                pred = model(inputs) # forward

efficientnet-phospho-B-15
3200
800


  0%|          | 0/20 [00:00<?, ?it/s]


RuntimeError: Given groups=32, weight of size [32, 1, 3, 3], expected input[64, 8, 134, 17] to have 32 channels, but got 8 channels instead

In [5]:
import re
import collections
import torch
from torch import nn
from utils import *
from torch.nn import functional as F

BlockArgs = collections.namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'stride', 'se_ratio'])

# Change namedtuple defaults
# GlobalParams.__new__.__defaults__ = (None,) * len(GlobalParams._fields)
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)

In [6]:
def efficientnet(width_coefficient=None, depth_coefficient=None, dropout_rate=0.2,
                 drop_connect_rate=0.2, image_size=None, num_classes=1):
    """ Creates a efficientnet model. """

    blocks_args = [
        'r1_k3_s11_e1_i8_o16_se0.25',
        'r1_k3_s22_e6_i16_o32_se0.25',
        'r1_k5_s22_e6_32_o64_se0.25',
        # 'r3_k3_s22_e6_i40_o80_se0.25',
        # 'r3_k5_s11_e6_i80_o112_se0.25',
        # 'r4_k5_s22_e6_i112_o192_se0.25',
        # 'r1_k3_s11_e6_i192_o320_se0.25',
        # 'r1_k3_s11_e6_i24_o48_se0.25',
    ]
    blocks_args = BlockDecoder.decode(blocks_args)

    global_params = GlobalParams(
        batch_norm_momentum=0.99,
        batch_norm_epsilon=1e-3,
        dropout_rate=dropout_rate,
        drop_connect_rate=drop_connect_rate,
        # data_format='channels_last',  # removed, this is always true in PyTorch
        num_classes=num_classes,
        width_coefficient=width_coefficient,
        depth_coefficient=depth_coefficient,
        depth_divisor=8,
        min_depth=None,
        image_size=image_size,
    )

    return blocks_args, global_params

blocks_args, global_params = efficientnet(width_coefficient=1.0, depth_coefficient=1.0, dropout_rate=0.7, image_size=[263, 15])

In [ ]:
def _encode_block_string(block):
    """Encodes a block to a string."""
    args = [
        "r%d" % block.num_repeat,
        "k%d" % block.kernel_size,
        "s%d%d" % (block.strides[0], block.strides[1]),
        "e%s" % block.expand_ratio,
        "i%d" % block.input_filters,
        "o%d" % block.output_filters
        # kernel_size is kernel size for convolution e.g. 3 x 3
        # num_repeat specifies how many times a particular block needs to be repeated, must be greater than zero
        # input_filters and output_filters are numbers of specified filters
        # expand_ratio is input filter expansion ratio
        # id_skip suggests whether to use skip connection or not
        # se_ratio provides squeezing ratio for squeeze and excitation block
    ]
    if 0 < block.se_ratio <= 1:
        args.append("se%s" % block.se_ratio)
    if block.id_skip is False:
        args.append("noskip")
    return "_".join(args)

In [7]:
import re
def _decode_block_string(block_string):
    """ Gets a block through a string notation of arguments. """
    assert isinstance(block_string, str)

    ops = block_string.split('_')
    options = {}
    for op in ops:
        splits = re.split(r'(\d.*)', op)
        if len(splits) >= 2:
            key, value = splits[:2]
            options[key] = value

    # Check stride
    # assert (('s' in options and len(options['s']) == 1) or
    #         (len(options['s']) == 2 and options['s'][0] == options['s'][1]))
    # assert (len(options['k']) < 4) and (len(options['s']) < 4), 'stride 또는 kernel_size를 조정하세요'
    # if len(options['k'])>=2:
    #     kernel_size = (int(options['k'][:-1]), int(options['k'][-1]))
    # else:
    #     kernel_size = (int(options['k']),int(options['k']))

    #
    # if len(options['s'])>=2:
    #     stride = (int(options['s'][:-1]), int(options['s'][-1]))
    # else:
    #     stride = (int(options['s']), int(options['s']))

    return BlockArgs(
        kernel_size=(int(options['kh']), int(options['kw'])),
        num_repeat=int(options['r']),
        input_filters=int(options['i']),
        output_filters=int(options['o']),
        expand_ratio=int(options['e']),
        id_skip=('noskip' not in block_string),
        se_ratio=float(options['se']) if 'se' in options else None,
        stride=(int(options['sh']), int(options['sw'])))

_decode_block_string('r1_kh2_kw2_sh2_sw1_e1_i32_o16_se0.25')

BlockArgs(kernel_size=(2, 2), num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, stride=(2, 1), se_ratio=0.25)

In [11]:
global_params, blocks_args

(GlobalParams(width_coefficient=1.0, depth_coefficient=1.0, image_size=[263, 15], dropout_rate=0.7, num_classes=1, batch_norm_momentum=0.99, batch_norm_epsilon=0.001, drop_connect_rate=0.2, depth_divisor=8, min_depth=None, include_top=None),
 [BlockArgs(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, id_skip=True),
  BlockArgs(num_repeat=2, kernel_size=3, stride=[2], expand_ratio=6, input_filters=16, output_filters=24, se_ratio=0.25, id_skip=True),
  BlockArgs(num_repeat=1, kernel_size=5, stride=[2], expand_ratio=6, input_filters=24, output_filters=40, se_ratio=0.25, id_skip=True)])

In [45]:
import math

class Conv2dNonPadding(nn.Conv2d):
    """ 2D Convolutions like TensorFlow, for a fixed image size"""

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, image_size=None, **kwargs):
        super().__init__(in_channels, out_channels, kernel_size, stride, **kwargs)
        # self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2

        # Calculate padding based on image size and save it
        assert image_size is not None
        ih, iw = image_size if type(image_size) == list else [image_size, image_size]
        kh, kw = self.weight.size()[-2:]
        sh, sw = self.stride if type(self.stride) == list else [self.stride, self.stride]
        # sh, sw = self.stride
        oh, ow = math.ceil(ih / sh), math.ceil(iw / sw)
        self.static_padding = nn.Identity()

    def forward(self, x):
        x = self.static_padding(x)
        x = F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [83]:
x = torch.randn((1, 1, 263, 15))
layer = Conv2dStaticSamePadding_(in_channels=1, out_channels=8, kernel_size=(15, 1), stride=(4, 2), image_size=(263, 15))

In [84]:
layer(x).shape

torch.Size([1, 8, 66, 8])

In [52]:
def calculate_output_image_size(input_image_size, stride):
    """Calculates the output image size when using Conv2dSamePadding with a stride.
       Necessary for static padding. Thanks to mannatsingh for pointing this out.

    Args:
        input_image_size (int, tuple or list): Size of input image.
        stride (int, tuple or list): Conv2d operation's stride.

    Returns:
        output_image_size: A list [H,W].
    """
    if input_image_size is None:
        return None
    image_height, image_width = get_width_and_height_from_size(input_image_size)
    sh, sw = stride if type(stride) == (list) or (tuple) else [stride, stride]
    # stride = stride if isinstance(stride, int) else stride[0]
    image_height = int(math.ceil(image_height / sh))
    image_width = int(math.ceil(image_width / sw))
    return [image_height, image_width]

In [55]:
bn_mom = 1 - global_params.batch_norm_momentum
bn_eps = global_params.batch_norm_epsilon
image_size = global_params.image_size
Conv2d = partial(Conv2dNonPadding, image_size=image_size)

In [68]:
# Stem

in_channels = 1
out_channels = round_filters(8, global_params)
print(out_channels)
_conv_stem = Conv2d(in_channels, out_channels, kernel_size=(15, 1), stride=(2, 1), bias=False)
_bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)
image_size = calculate_output_image_size(image_size, stride=(2, 1))

32
<class 'torch.nn.parameter.Parameter'>
torch.Size([32, 1, 15, 1])
15 1


In [63]:
block_args = blocks_args[0]
input_filters=round_filters(block_args.input_filters, global_params)

In [64]:
input_filters

32

In [9]:
def encode_block_string(block):
    """Encodes a block to a string."""
    args = [
        "r%d" % block.num_repeat,
        "k%d" % block.kernel_size,
        "s%d%d" % (block.strides[0], block.strides[1]),
        "e%s" % block.expand_ratio,
        "i%d" % block.input_filters,
        "o%d" % block.output_filters
        # kernel_size is kernel size for convolution e.g. 3 x 3
        # num_repeat specifies how many times a particular block needs to be repeated, must be greater than zero
        # input_filters and output_filters are numbers of specified filters
        # expand_ratio is input filter expansion ratio
        # id_skip suggests whether to use skip connection or not
        # se_ratio provides squeezing ratio for squeeze and excitation block
    ]
    if 0 < block.se_ratio <= 1:
        args.append("se%s" % block.se_ratio)
    if block.id_skip is False:
        args.append("noskip")
    return "_".join(args)

In [11]:
def decode(string_list):
    """
    Decodes a list of string notations to specify blocks inside the network.
    :param string_list: a list of strings, each string is a notation of block
    :return: a list of BlockArgs namedtuples of block args
    """
    assert isinstance(string_list, list)
    blocks_args = []
    for block_string in string_list:
        blocks_args.append(_decode_block_string(block_string))
    return blocks_args

In [12]:
def _decode_block_string(block_string):
    """Gets a block through a string notation of arguments."""
    assert isinstance(block_string, str)

    ops = block_string.split("_")
    options = {}
    for op in ops:
        splits = re.split(r"(\d.*)", op)
        if len(splits) >= 2:
            key, value = splits[:2]
            options[key] = value

    # # Check stride
    # assert ("s" in options and len(options["s"]) == 1) or (
    #     len(options["s"]) == 2 and options["s"][0] == options["s"][1]
    # )

    return BlockArgs(
        kernel_size=(int(options["kh"]), int(options["kw"])),
        num_repeat=int(options["r"]),
        input_filters=int(options["i"]),
        output_filters=int(options["o"]),
        expand_ratio=int(options["e"]),
        id_skip=("noskip" not in block_string),
        se_ratio=float(options["se"]) if "se" in options else None,
        stride=(int(options["sh"]), int(options["sw"])),
    )

In [14]:
blocks_args = [
    "r1_kh3_kw1_sh1_sw1_e1_i32_o16_se0.25",
    # "r2_k3_s22_e6_i16_o24_se0.25",
    # "r1_k5_s22_e6_i24_o40_se0.25",
    # 'r3_k3_s22_e6_i40_o80_se0.25',
    # 'r3_k5_s11_e6_i80_o112_se0.25',
    # 'r4_k5_s22_e6_i112_o192_se0.25',
    # 'r1_k3_s11_e6_i192_o320_se0.25',
    # 'r1_k3_s11_e6_i24_o48_se0.25',
]
blocks_args = decode(blocks_args)

In [ ]:
def _encode_block_string(block):
    """Encodes a block to a string."""
    args = [
        "r%d" % block.num_repeat,
        "kh%d" % block.kernel_size[0],
        "s%d%d" % (block.strides[0], block.strides[1]),
        "e%s" % block.expand_ratio,
        "i%d" % block.input_filters,
        "o%d" % block.output_filters
        # kernel_size is kernel size for convolution e.g. 3 x 3
        # num_repeat specifies how many times a particular block needs to be repeated, must be greater than zero
        # input_filters and output_filters are numbers of specified filters
        # expand_ratio is input filter expansion ratio
        # id_skip suggests whether to use skip connection or not
        # se_ratio provides squeezing ratio for squeeze and excitation block
    ]
    if 0 < block.se_ratio <= 1:
        args.append("se%s" % block.se_ratio)
    if block.id_skip is False:
        args.append("noskip")
    return "_".join(args)

In [ ]:
def encode(blocks_args):
    """
    Encodes a list of BlockArgs to a list of strings.
    :param blocks_args: a list of BlockArgs namedtuples of block args
    :return: a list of strings, each string is a notation of block
    """
    block_strings = []
    for block in blocks_args:
        block_strings.append(_encode_block_string(block))
    return block_strings

In [20]:
block = blocks_args[0]

In [26]:
def _encode_block_string(block):
    """Encodes a block to a string."""
    args = [
        "r%d" % block.num_repeat,
        "kh%d" % block.kernel_size[0],
        "kw%d" % block.kernel_size[1],
        "sh%d" % block.stride[0],
        "sw%d" % block.stride[1],
        # "s%d%d" % (block.strides[0], block.strides[1]),
        "e%s" % block.expand_ratio,
        "i%d" % block.input_filters,
        "o%d" % block.output_filters
        # kernel_size is kernel size for convolution e.g. 3 x 3
        # num_repeat specifies how many times a particular block needs to be repeated, must be greater than zero
        # input_filters and output_filters are numbers of specified filters
        # expand_ratio is input filter expansion ratio
        # id_skip suggests whether to use skip connection or not
        # se_ratio provides squeezing ratio for squeeze and excitation block
    ]
    if 0 < block.se_ratio <= 1:
        args.append("se%s" % block.se_ratio)
    if block.id_skip is False:
        args.append("noskip")
    return "_".join(args)

In [27]:
_encode_block_string(blocks_args[0])

'r1_kh3_kw1_sh1_sw1_e1_i32_o16_se0.25'

In [59]:
blocks = nn.ModuleList([])
for block_args in blocks_args:

    # Update block input and output filters based on depth multiplier.
    block_args = block_args._replace(
        input_filters=round_filters(block_args.input_filters, global_params),
        # input_filters=8,
        output_filters=round_filters(block_args.output_filters, global_params),
        num_repeat=round_repeats(block_args.num_repeat, global_params)
    )

    # The first block needs to take care of stride and filter size increase.
    self._blocks.append(MBConvBlock(block_args, self._global_params, image_size=image_size))
    image_size = calculate_output_image_size(image_size, block_args.stride)
    if block_args.num_repeat > 1:  # modify block_args to keep same output size
        block_args = block_args._replace(input_filters=block_args.output_filters, stride=1)
    for _ in range(block_args.num_repeat - 1):
        self._blocks.append(MBConvBlock(block_args, self._global_params, image_size=image_size))

In [61]:
blocks_args

[BlockArgs(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=2, kernel_size=3, stride=[2], expand_ratio=6, input_filters=16, output_filters=24, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=1, kernel_size=5, stride=[2], expand_ratio=6, input_filters=24, output_filters=40, se_ratio=0.25, id_skip=True)]